# Parse KB/MKC generated (faulty/non-compliant) ALTO XML to get OCR text. 
Results of research (only tried on 1 file but probably systematic) – the ALTO XML provided seems faulty/non-compliant. Had to replace <alto> namespace declarations and change the tag to lowercase. I [used this sample](https://raw.githubusercontent.com/altoxml/reference_samples/master/v2/sbb/newspapers/27971740_1890-04-01_38_077_0_001.xml) to find the correct code.

Detta fixar filen:

<Alto>...</Alto> => <alto>...</alto> [tolowercase]

Denna header (de 2 första raderna innan <description>) ska ersätta kbs/MKCs:

```xml
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<alto xmlns="http://www.loc.gov/standards/alto/ns-v2#" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.loc.gov/standards/alto/ns-v2# http://www.loc.gov/standards/alto/alto-v2.0.xsd">
```

Även den avslutande </Alto> tagen måste göras till lowercase: </alto>. Uppdatering – det är bara alto tagen som måste ersättas/korrigeras, <?xml ...?> funkar.

#### Parse from file alto.xml

In [9]:
#!/usr/bin/env python
# Usage: python alto_ocr_text.py <altofile>

import codecs
import os
import sys
import xml.etree.ElementTree as ET

altofile = 'alto.xml'


if sys.stdout.encoding != 'UTF-8':
    sys.stdout = codecs.getwriter('utf-8')(sys.stdout.buffer, 'strict')

namespace = {'alto-1': 'http://schema.ccs-gmbh.com/ALTO',
             'alto-2': 'http://www.loc.gov/standards/alto/ns-v2#',
             'alto-3': 'http://www.loc.gov/standards/alto/ns-v3#',
             'alto-mkc': 'http://www.loc.gov/standards/alto/alto-v2.0.xsd'
             }
#tree = ET.parse(sys.argv[1])
tree = ET.parse(altofile)
xmlns = tree.getroot().tag.split('}')[0].strip('{')
if xmlns in namespace.values():
    for lines in tree.iterfind('.//{%s}TextLine' % xmlns):
        sys.stdout.write('\n')
        for line in lines.findall('{%s}String' % xmlns):
            text = line.attrib.get('CONTENT') + ' '
            sys.stdout.write(text)
else:
    print('ERROR: Not a valid ALTO file (namespace declaration missing)')



N :o I 
Måndagen 
I dag finnes kunsjorclt följande 
eanco-lån på effekter 
.r \icls .jern får i Banken samt Line-Conto 
vet i Götheborg år i83i belånas till följan 
,Ie priser för Skeppuudet För Upland och 
Roslagen 5 R :dr Korbergs Skinsbergs 
Linden Nors Ramsbergs Lekebergs Ler 
bäcks Hjulsjö Carlskoga Philipstads 
Söderberkes Satsens och Floda Ostcrbtrgs 
Lag Södermanlands orh (jstcrgötlilands Liin 
Tabergs Bergslag och Jönköpings Län Of 
&gt;smsj &lt;5 och Torsåker samt Masngnarne i 
Gefleborgs Westernorrlands Norrbottens 
facb Westerbottens Liin 4 B :dr 16 sk 
Hjulsjö Grythytte och Helleforss Bergsl 
Norrberkes och Tuna Grangärdes Öster 
dals Kalmar och Kronobergs Län 4 
R :dr 

Hellbo Langvinds Gnarps Galtström 
Lögdö B ^llsta och Olofsforss masugnar a 
R :dr 16 sk 
Underrättelse för lånsökande 4 § i inom 
Kocgl Förordn d i-j Maji iö14- 
Län på andra effekter att betalas d 3i 
Oct l83a men eljest med bankens rätt till 
» 'v veckors uppsägning när som helst B än tan 
4 p c i Skep

#### Parse from url

This script requsts a url to an alto-XML file corrects the faulty XML and parses it to get ocr-text. 

In [32]:
import codecs
import os
import sys
import xml.etree.ElementTree as ET
from urllib.request import urlopen

altourl = 'https://data.kb.se/dark-38655/bib4345612_18310103_0_s_0001_alto.xml'
altourl = 'https://data.kb.se/dark-32256/bib4345612_18630811_0_s_0003_alto.xml'


if sys.stdout.encoding != 'UTF-8':
    sys.stdout = codecs.getwriter('utf-8')(sys.stdout.buffer, 'strict')

namespace = {'alto-1': 'http://schema.ccs-gmbh.com/ALTO',
             'alto-2': 'http://www.loc.gov/standards/alto/ns-v2#',
             'alto-3': 'http://www.loc.gov/standards/alto/ns-v3#',
             'alto-mkc': 'http://www.loc.gov/standards/alto/alto-v2.0.xsd'
             }
#tree = ET.parse(sys.argv[1])

with urlopen(altourl) as f:

    ## THIS CODE PATCHES THE XML CODE BEFORE PARSING
    corrected_tag = b'<alto xmlns="http://www.loc.gov/standards/alto/ns-v2#" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.loc.gov/standards/alto/ns-v2# http://www.loc.gov/standards/alto/alto-v2.0.xsd">\r\n'
    alto_list = f.readlines()
    #print("Line to replace: ", alto_list[1])
    alto_list[1]=corrected_tag
    #print("Corrected line: ", alto_list[1])
    #print("Line to replace: ", alto_list[-1])
    alto_list[-1] = b'</alto>\r\n'
    #print("Corrected line: ", alto_list[-1])
    ## ## ##
    
    tree = ET.ElementTree(ET.fromstringlist(alto_list))
    xmlns = tree.getroot().tag.split('}')[0].strip('{')
    if xmlns in namespace.values():
        for lines in tree.iterfind('.//{%s}TextLine' % xmlns):
            sys.stdout.write('\n')
            for line in lines.findall('{%s}String' % xmlns):
                text = line.attrib.get('CONTENT') + ' '
                sys.stdout.write(text)
    else:
        print('ERROR: Not a valid ALTO file (namespace declaration missing)')



rott deraf att de seglande varit mindre kun 
niga i manövreringen De drunknade voro 
alfa i sina bästa år och efterlemna 1 'et 
tersson samt Carl Ericsson hustru och barn 
— Ombud från Hambro .tfe son i London 
från privatbanken i Köpenhamn samt från 
bankirhuset Suhr son dersammastädes 
vistas i Christiania förmodligen säger Mor 
genbladet för att inleda underhandlingar 
rörande det af stortinget niedgifna statslå 
net på 1V3 mill spd 
— Djurgårdsteatern kommer att rifvas 
sedan hr IloosJ föreställningar si utåt med 
September månads utgång 
— Hr Selinders sällskap slutar sina fö 
reställningar på Klingenbergs teater i Chri 
Btiania dea 23 dennes 
— Utstakningen af jernvägslinien å nord 
vestra stambanan från Porla till Christine 
hamn pågår nu som bäst En del förbe 
redande arbeten såsom rödjning och berg 
sprängning lä :a komma att börjas redan 
denna höst 
&gt;— Det bekräf ar sig att några svenska 
ynglingar blifvit af rjssarne tillfångatagna 
och dömda att sändas till de siberisk